In [6]:
from numericMethods.ode.run_kut4 import integrate
from numericMethods.rootFinding.ridders2 import ridder
import numpy as np
from math import isnan



#Define the initial conditions as a function of u - in the shooting method we don't know the proper initial condition for E(0)
#such that the condition phi(d)=V0, where d is the laminar flux region.


#Define the initial conditions as a function of u - in the shooting method we don't know the proper initial condition for E(0)
#such that the condition phi(d)=V0, where d is the laminar flux region.
Psi_all = []
Cp_all = []
Cm_all = []
E_all = []

reactRate = [0, 7.78e-8, 7.76e-6]

for p in range(0,len(reactRate)):

    def init(u):
        #boundary conditions: 
        #at x = delta:
        return np.array([Cb, Cb, 0, u[0], u[1], u[2]])

    # Define the function F(x,y) such that y' = F(x,y)
    def F(x,y):
        #C plus
        Cp =  y[0]
        #C minus
        Cm = y[1]
        #E electric field
        V = y[2]
        #concentration gradient
        DCp = y[3]
        DCm = y[4]
        #Electric potential
        E = y[5]
        #Function
        #r0 = r/k
        F = np.zeros(6)
 
        F[0] = DCp*E + Cp * (Cp - Cm)
        F[1] = -DCm*E - Cm * (Cp - Cm)
        F[2] = (Cp - Cm)
        F[3] = DCp
        F[4] = DCm
        F[5] = -E   
        return F

    def invert(A):
        B = []
        N = len(A)
        for i in range(0,N):
            B.append(A[N-1-i])
        return np.array(B)


    def find_not_nan(L):
        for i in range(0,len(L)-1):
            if(isnan(L[len(L)-1-i]) == False):
                return len(L)-1-i
        return 0

    def r(u):
        X,Y = integrate(F, xStart, init(u), xStop, step)
        V_u = Y[-1,2]
        V1_u = Y[-2,2]
        rho1_u = Y[-2,1]
        rho2_u = Y[-2,0]
        #print(Y)
        if(isnan(rho1_u)):
            index1 = int(find_not_nan(Y[:,2]))
            rho1_u = Y[index1,0]
        if(isnan(rho2_u)):
            index2 = int(find_not_nan(Y[:,1]))
            rho2_u = Y[index2,1]
        if(isnan(V_u)):
            index3 = int(find_not_nan(Y[:,0]))
            V_u = Y[index3,2]
            V1_u = Y[index3-1,2]
        
        
        
        
        kf = 0.1
        g1 = 1 / (1 - (V1_u - V_u) + kf) 
        g2 = 1 / (1 + (V1_u - V_u) ) 
        
        return np.array([rho1_u * ((V1_u - V_u) / h ) + kf, -rho1_u * ((V1_u - V_u) / h ), (V_u-V_0)/V_0])

    #Here we start the algorithm to find de numeric solution to the system defined by y'=F(x,y). Note that here y
    #is a vector with four components

    #Define integration range.

    xStop = 20.0
    xStart = 0.0

    #Define the number of steps of integration.
    N = 1E3
    step = (xStop-xStart)/N
    h = step
    r0 = reactRate[p]
    u1 = [1e-5, 1e-5, 1e-5]
    u2 = [0, 0, 0]

    z = 2
    e = 1.60217662E-19
    k = 1.38064852E-23
    T = 300
    Na = 6.02E23
    Fa = 96485.3329#Na * e
    R = Na * k
    coef = z*e/(k*T)
    V_0 = -coef*0.15
    D = 1.05
    Cb = 1
    print('r(b): ' + str(r(u1)))
    print('r(a): ' + str(r(u2)))
    epsilon = 80.9 * 8.85418782E-12
    k = np.sqrt(2 * Cb * (z*Fa) ** 2 / (R * T * epsilon))
    u = ridder(r, u1, u2)

    #Plot results, if initial condition for E(0) is found.


    import matplotlib.pyplot as plt

    if (u != None):
        X,Y = integrate(F, xStart, init(u), xStop, step)
        Cp = invert(Y[:,0])
        Cm = invert(Y[:,1])
        E = 1/coef * invert(Y[:,2])
        phi = 1/coef * invert(Y[:,3])
        xi = X
        print("Border condition at interface is: " + str(-77.357*0.15))
        print("Obtained border condition with shooting method: " + str(phi[0]))
        print("Obtained border condition in the bulk: " + str(E[-1]))
        print("Shooting error: " + str((phi[0]-V_0)/V_0))
        

    #Plot electric potential
        #plt.figure(1)
        #plt.title('Numeric Potential', fontsize=16, fontweight='bold')
        #plt.xlabel(r'$\xi = \kappa x$', fontsize=16)
        #plt.ylabel(r'Dimentionless potential ', fontsize=16)
        #plt.plot(X[:], phi, 'g', label = "Potential")
        #plt.axis([xStart, xStop, -20, -0.0])
        #plt.legend()
        #plt.savefig('potential.eps', format='eps', dpi=1000, fontsize=16, fontweight='bold')
        #plt.show()
        
        #Plot electric field
        #plt.figure(2)
        #plt.plot(X[:], E, 'g', label = "Electric Field")
        #plt.axis([xStart, xStop, 0.0, 100])
        #plt.legend()
        #plt.savefig('Efield.eps', format='eps', dpi=1000, fontsize=16, fontweight='bold')
        #plt.show()

        #Plot Concetration
        #plt.figure(3)
        #plt.title('Numeric Concentration', fontsize=16, fontweight='bold')
        #plt.xlabel(r'$\xi = \kappa x$', fontsize=16)
        #plt.ylabel(r'Molar Concentration ', fontsize=16)
        #plt.plot(X[:], Cp, 'b', label = "Concentration +")
        #plt.axis([xStart, xStop, 0, 1])
        #plt.plot(X[:], Cm, 'r', label = "Concentration -")
        #plt.legend()
        
        

        file = open('./results/potential-num-r'+str(r0)+'.txt', 'w')
        file2 = open('./results/cp-num-r'+str(r0)+'.txt', 'w')
        file3 = open('./results/cm-num-r'+str(r0)+'.txt', 'w')
        file4 = open('./results/E-num-r' + str(r0) + '.txt', 'w')


        for i in range(0, len(xi)):
            file.write(str(xi[i]/k*1E9) + "\t" +  str(phi[i]) + "\n")
            file2.write(str(xi[i]/k*1E9) + "\t" +  str(Cp[i]) + "\n")
            file3.write(str(xi[i]/k*1E9) + "\t" +  str(Cm[i]) + "\n")
            file4.write(str(xi[i]/k*1E9) + "\t" + str(E[i]) + "\n")
            
        file.close()
        file2.close()
        file3.close()
        file4.close()
        Psi_all.append(phi)
        E_all.append(E)
        Cp_all.append(Cp)
        Cm_all.append(Cm)

        


/Users/creative-work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in double_scalars
/Users/creative-work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in double_scalars
/Users/creative-work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
/Users/creative-work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars
/Users/creative-work/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars


r(b): [-inf  inf -inf]
r(a): [ 0.1 -0.  -1. ]
fa: [-inf  inf -inf]
fb: [ 0.1 -0.  -1. ]
fc: [-1.20327465e+290  1.20327465e+290 -1.72504015e+094]


ValueError: cannot convert float NaN to integer

In [ ]:
X = X / k*1E9 
mw = 4
fs = 30
plt.figure(1)
plt.title('', fontsize=fs, fontweight='bold')
plt.xlabel(r'x', fontsize=fs)
plt.ylabel(r'Concentracion [$M$]', fontsize=fs)
plt.plot(X, Cp_all[0], 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, Cp_all[1], 'r*', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, Cp_all[2], 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.ylim(0,10)
plt.legend(prop={'size': 12})
plt.show()

#plt.savefig('potential.eps', format='eps', dpi=1000, fontsize=16, fontweight='bold')

In [ ]:
plt.savefig('Concentration-num-'+str(r0)+'.eps', format='eps', dpi=1000, fontsize=16, fontweight='bold')

In [ ]:

plt.figure(1)
plt.title('', fontsize=fs, fontweight='bold')
plt.xlabel(r'x', fontsize=fs)
plt.ylabel(r'Concentracion [$M$]', fontsize=fs)
plt.plot(X, Cm_all[0], 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, Cm_all[1], 'r*', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, Cm_all[2], 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.ylim(0,1.1)
plt.legend(prop={'size': 12})
plt.show()


In [ ]:

plt.figure(1)
plt.title('', fontsize=fs, fontweight='bold')
plt.xlabel(r'x', fontsize=fs)
plt.ylabel(r'Electric Field [$M$]', fontsize=fs)
plt.plot(X, E_all[0], 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, E_all[1], 'r*', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, E_all[2], 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.legend(prop={'size': 12})
plt.show()


In [ ]:
plt.figure(1)
plt.title('', fontsize=fs, fontweight='bold')
plt.xlabel(r'x', fontsize=fs)
plt.ylabel(r'Potential', fontsize=fs)
plt.plot(X, Psi_all[0], 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, Psi_all[1], 'ro', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(X, Psi_all[2], 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.legend(prop={'size': 12})
plt.show()

In [ ]:

def remove_points(A):
    A = np.delete(A, [1, 2, 3])
    for i in range(0,int(len(A)/4)):
        index = i+4
        A = np.delete(A, [index-2, index-1, index])
    return A

plt.figure(figsize=(20,16))
plt.suptitle('Numeric Analysis of Steady State', fontsize=fs*1.5)
plt.subplot(2,2,1)
plt.title(r'(a)', fontsize=fs, fontweight='bold')
plt.ylabel(r'Concentracion [$Mol/m^3$]', fontsize=fs)

plt.plot(remove_points(X), remove_points(Cp_all[0]), 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(Cp_all[1]), 'rs', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(Cp_all[2]), 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.ylim(0,2.5)
#plt.legend(prop={'size': 12})

plt.subplot(2,2,2)
plt.title('(b)', fontsize=fs, fontweight='bold')

plt.plot(remove_points(X), remove_points(Cm_all[0]), 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(Cm_all[1]), 'rs', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(Cm_all[2]), 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.ylim(0,1.1)
#plt.legend(prop={'size': 12})

plt.subplot(2,2,3)
plt.title(r'(c)', fontsize=fs, fontweight='bold')
plt.xlabel(r'x [nm]', fontsize=fs)
plt.ylabel(r'Electric Field [$V/m$]', fontsize=fs)
plt.plot(remove_points(X), remove_points(E_all[0]), 'b--', linewidth=mw)#, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(E_all[1]), 'rs', markersize=mw)#, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(E_all[2]), 'g^', markersize=mw)#, markersize=4, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
#plt.legend(prop={'size': 12})

plt.subplot(2,2,4)
plt.title(r'(d)', fontsize=fs, fontweight='bold')
plt.xlabel(r'x [nm]', fontsize=fs)
plt.ylabel(r'Potential [$V$]', fontsize=fs)
plt.plot(remove_points(X), remove_points(Psi_all[0]), 'b--', linewidth=mw, label='r = '+str(reactRate[0])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(Psi_all[1]), 'rs', markersize=mw, label='r = '+str(reactRate[1])+r'$\frac{Mol}{m^3s}$')
plt.plot(remove_points(X), remove_points(Psi_all[2]), 'g^', markersize=mw, label='r = '+str(reactRate[2])+r'$\frac{Mol}{m^3s}$')
plt.legend(prop={'size': 24})

plt.savefig('../../../img/results-numeric.eps', format='eps', dpi=1000, fontsize=fs, fontweight='bold')

plt.show()



In [ ]:
import numpy as np
from numericMethods.rootFinding.newtonRaphson2 import newtonRaphson2

params0 = {
            'bulkConcentration': 100, 
            'diffusionCoefficientScale': 1e-9, 
            'laminarFlowRegion': 1e-10, 
            'reactionRate': 1e5,
            "z": 2,
            "e": 1.60217662E-19,
            "kb": 1.38064852E-23,
            "T": 300,
            "Na": 6.02E23,
            "Fa": 96485.3329, #Na * e
            "R": 8.314472,
            "V0": -0.15,
            "D1": 1.07,
            "D2":  0.733,
            "epsilon": 80.9 * 8.85418782E-12,
            "length": 5.0
}

calculated_parameters = {
    'kappa': np.sqrt((params0['Fa'] * params0['z']) ** 2 * params0['bulkConcentration'] / (params0['epsilon'] * params0['R'] * params0['T'])),
    'Psi0': (params0['Fa'] * params0['z'] * params0['V0'] ) / (params0['R'] * params0['T'])
}




def residual(y):  # Residuals of finite diff. Eqs. (8.11)
    r = np.zeros([3, m + 1])
    R = np.zeros(3*(m + 1))
    
    index_Cp = 0
    index_Cm = m + 1
    index_Psi = 2 * (m + 1)
    indxs = [index_Cp, index_Cm, index_Psi]
    
    #Boundary Conditions
    dxi = (xStop - xStart)/m
    
    kappa = calculated_parameters['kappa']
    Psi0 = calculated_parameters['Psi0']
    Cb = params0['bulkConcentration']
    D1 = params0['D1'] * params0['diffusionCoefficientScale']
    kf = params0['reactionRate']
    k2 = kf/( D1 * kappa * Cb)
    g1 = 1 / (1 - (y[index_Cp + 1] - y[index_Cp + 0]) + k2)
    g2 = 1 / (1 + (y[index_Cm + 1] - y[index_Cp + 0]))
    
 
    y[index_Cp] = g1 * y[index_Cp + 1]
    y[index_Cm] = g2 * y[index_Cp + 1]
    y[index_Psi] = Psi0
    y[index_Cm - 1] = Cb
    y[index_Psi - 1] = Cb
    y[index_Psi + m ] = 0
    
    '''
    r[0, 1] = g1 * y[0, 1] - 2.0*y[0, 1] + y[0, 2] - h*h*F(x[1],y[0, 1],(y[0, 2] - y[0, 0])/(2.0*h))[0]
    r[1, 1] = g2 * y[1, 1] - 2.0*y[1, 1] + y[1, 2] - h*h*F(x[i],y[1, i],(y[1, i+1] - y[1, i-1])/(2.0*h))[0]
    r[2, 1] = V_0 - 2.0*y[0, 1] + y[0, 2] - h*h*F(x[i],y[2, i],(y[2, i+1] - y[2, i-1])/(2.0*h))[0]
    r[0, m-1] = - h*h*F(x[i],y[0, i],(y[0, i+1] - y[0, i-1])/(2.0*h))[0]
    r[1, m-1] = - h*h*F(x[i],y[1, i],(y[1, i+1] - y[1, i-1])/(2.0*h))[0]
    r[2, m-1] = - h*h*F(x[i],y[2, i],(y[2, i+1] - y[2, i-1])/(2.0*h))[0]
    '''
    
    for j in range(2):
        for i in range(0, m):
            ind = indxs[j] 
            DY = [(y[index_Cp + i+1] - y[index_Cp + i-1])/(2.0*h), (y[index_Cm + i+1] - y[index_Cm + i-1])/(2.0*h), (y[index_Psi + i+1] - y[index_Psi + i-1])/(2.0*h)]
            Y = [y[index_Cp + i], y[index_Cm + i], y[index_Psi + i]] 
            R[i + ind] = y[ind + i-1] - 2.0*y[ind + i] + y[ind + i+1] - h*h*F(x[i],Y,DY)[j]

    
        #y[i-1] - 2.0*y[i] + y[0, i+1] + y[1, i-1] - 2.0*y[1, i] + y[1, i+1] + y[2, i-1] - 2.0*y[2, i] + y[2, i+1] - h*h*F(x[i],y[:, i],(y[:, i+1] - y[:, i-1])/(2.0*h))[0] - h*h*F(x[i],y[:, i],(y[:, i+1] - y[:, i-1])/(2.0*h))[1]  - h*h*F(x[i],y[:, i],(y[:, i+1] - y[:, i-1])/(2.0*h))[2]
    
    return R



def F(x,y, yPrime):
    #C plus
    Cp =  y[0]
    #C minus
    Cm = y[1]
    #E electric field
    V = y[2]
    #concentration gradient
    DCp = yPrime[0]
    DCm = yPrime[1]
    #Electric potential
    E = -yPrime[2]
    #Function
    #r0 = r/k
    F = np.zeros(3)
    F[0] = DCp * E + Cp * (Cp - Cm)
    F[1] = -DCm * E - Cm * (Cp - Cm)
    F[2] = (Cp - Cm)
    return F

def startSoln(x):       # Starting solution y(x)
    y = np.zeros([3 * (m + 1)])
    for i in range(len(y)-1): 
            y[i] = 0.5*x[i%m]
    print(y)
    return y



xStart = 0.0
xStop = 3.0
m = 10
h = (xStop - xStart)/m
x = np.arange(xStart,xStop + h,h)
print(x)
y = newtonRaphson2(residual,startSoln(x),1.0e-5)

print("\n        x              y")
for i in range(m + 1):
    print("%14.5e %14.5e" %(x[i],y[i]))
#input("\nPress return to exit")